In [ ]:
# PyTorch implementation of the TensorFlow/Keras LSTM model from `LSTM_New.ipynb`
# -------------------------------------------------------------
# Requirements: pandas, numpy, torch (>=2.0 recommended)
# -------------------------------------------------------------

import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

# ---------------------------
# Hyper‑parameters & paths
# ---------------------------
BATCH_SIZE = 64
SEQ_LEN = 34          # timesteps per sample
INPUT_SIZE = 1        # one feature per timestep (after reshape)
HIDDEN_SIZE = 64
NUM_CLASSES = 2       # Win / Non‑Win one‑hot originally → convert to class index 0/1
EPOCHS = 20
LR = 1e-3
TRAIN_CSV = "allAtt_onehot_large_train_new8.csv"
TEST_CSV  = "allAtt_onehot_large_test_new8.csv"
MODEL_PATH = "lstm_pytorch.pth"

# ---------------------------
# Data loading & preprocessing
# ---------------------------
print("Loading data …")
train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)

# 34 特征列（与原 notebook 保持一致）
X_train = train_df.iloc[:, 4:38].values.astype(np.float32).reshape(-1, SEQ_LEN, INPUT_SIZE)
X_test  = test_df.iloc[:, 4:38].values.astype(np.float32).reshape(-1, SEQ_LEN, INPUT_SIZE)

# one‑hot → 单一标签
y_train = train_df.iloc[:, 38:].values.astype(np.float32).argmax(axis=1)
y_test  = test_df.iloc[:, 38:].values.astype(np.float32).argmax(axis=1)

train_ds = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
val_ds   = TensorDataset(torch.from_numpy(X_test),  torch.from_numpy(y_test))

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)

# ---------------------------
# Model definition
# ---------------------------
class LSTMClassifier(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_classes: int):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.bn   = nn.BatchNorm1d(hidden_size)
        self.fc   = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (batch, seq_len, input_size)
        _, (h_n, _) = self.lstm(x)     # h_n: (1, batch, hidden)
        h_n = h_n.squeeze(0)           # (batch, hidden)
        h_n = self.bn(h_n)             # BatchNorm over features
        logits = self.fc(h_n)
        return logits

# ---------------------------
# Training setup
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = LSTMClassifier(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# ---------------------------
# Training loop
# ---------------------------
for epoch in range(1, EPOCHS + 1):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    train_loss = running_loss / total
    train_acc  = correct / total

    # Validation
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            preds = outputs.argmax(dim=1)
            val_correct += (preds == y_batch).sum().item()
            val_total   += y_batch.size(0)
    val_acc = val_correct / val_total

    print(f"Epoch {epoch:2d}/{EPOCHS} | loss {train_loss:.4f} | train_acc {train_acc:.4f} | val_acc {val_acc:.4f}")

# ---------------------------
# Save model
# ---------------------------
torch.save(model.state_dict(), MODEL_PATH)
print(f"Model saved to {MODEL_PATH}")
